In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

from sklearn import linear_model, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
full_df = pd.read_csv('../../data/full_df.csv')
# train_df = pd.read_csv('../../data/train_df.csv')
# prior_df = pd.read_csv('../../data/prior_df.csv')
# test_df = pd.read_csv('../../data/test_df.csv')

In [3]:
full_df.drop('Unnamed: 0',axis=1,inplace=True)
full_df.head()

,user_id,product_id,eval_set,target,order_id,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,112108,49302,train,1,1,3,1,4,2.000000,170,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
1,112108,11109,train,1,1,3,1,4,3.333333,4579,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
2,112108,43633,train,1,1,3,1,4,4.000000,663,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
3,112108,22035,train,1,1,3,1,4,6.000000,61241,...,0.764706,4,167.0,9.823529,15.0,7.0,6.250000,0.750000,0.750000,0
4,79431,19660,train,1,36,16,2,23,4.562500,57921,...,0.500000,23,3238.0,17.597826,30.0,2.0,8.391304,0.695652,0.727273,0


In [4]:
#filling in null values - for previous calculations where user didn't do any reorders
full_df.product_ordered_twice_vol.fillna(0,inplace=True)
full_df.product_order_twice_ratio.fillna(0,inplace=True)
full_df.user_unique_reorder_count.fillna(0,inplace=True)
full_df.user_unique_reorder_perc.fillna(0,inplace=True)
full_df.user_total_items_after_first_order.fillna(0,inplace=True)
full_df.user_reorder_ratio.fillna(0,inplace=True)
full_df.isnull().sum()

user_id                                   0
product_id                                0
eval_set                                  0
target                                    0
order_id                                  0
user_product_count                        0
user_product_first_order                  0
user_product_last_order                   0
user_product_avg_basket_placement         0
product_ordered_vol                       0
product_reordered_vol                     0
product_ordered_once_vol                  0
product_ordered_twice_vol                 0
product_order_twice_ratio                 0
product_avg_reorders                      0
product_overall_reorder_prob              0
user_total_products                       0
user_total_products_reordered             0
user_unique_product_count                 0
user_unique_reorder_count                 0
user_unique_product_perc                  0
user_unique_reorder_perc                  0
user_total_items_after_first_ord

In [5]:
print(full_df.target.value_counts())
print(full_df.target.value_counts(normalize=True))


0    7645837
1     828824
Name: target, dtype: int64
0    0.9022
1    0.0978
Name: target, dtype: float64


## Splitting Data
Will first split dataset into train, test, validation sets and then do a T/T/S on training set for model tuning

In [6]:
full_df.shape

(8474661, 33)

In [7]:
np.random.seed(50)
train, validate, test = \
              np.split(full_df.sample(frac=1, random_state=42), 
                       [int(.8*len(full_df)), int(.9*len(full_df))])

In [8]:
print(train.shape) #80%
print(validate.shape) #10%
print(test.shape) #10%

(6779728, 33)
(847466, 33)
(847467, 33)


In [9]:
X= train.drop(["target",'eval_set'],axis=1)
y= train['target']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [24]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

## Logistic Regression 

In [19]:
1-y_train.mean()

0.9020504511427635

In [12]:
from sklearn.linear_model import LogisticRegression

In [27]:
logreg = LogisticRegression(solver='lbfgs', max_iter=400)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
# print(roc_auc_score(y_test, y_pred))
print(logreg.score(X_test, y_test))

0.9965817222809757


In [26]:
logit_simple = linear_model.LogisticRegression(C=1e9).fit(X_train, y_train)
print(np.mean(y_test == logit_simple.predict(X_test)))
logit_pred_proba = logit_simple.predict_proba(X_test)[:,1]

cm = pd.DataFrame(metrics.confusion_matrix
                  (y_true=y_test, y_pred=logit_pred_proba > .5), 
                  columns=['Negative Pred', 'Positive Pred'])


/Users/tiffany.chen/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9965706598935355


In [ ]:
tn = cm.iloc[0,0]
fn = cm.iloc[1,0]
fp = cm.iloc[0,1]
tp = cm.iloc[1,1]

accuracy = (tp+tn) / (tp+tn+fp+fn)
sensitivity = (tp) / (tp+fn)
precision= tp / (tp+fp) #proportion of the data points our model says was relevant actually were relevant
recall= tp/(tp+fn) #How many positives were predicted correctly
ppv = tp / (tp+fp)

## Feature Selection 

In [ ]:
sns.heatmap(full_df.corr(),vmin=-1, vmax=1, center=0,
           cmap=sns.diverging_palette(20, 220, n=200))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
X_model_new = SelectKBest(score_func=chi2,k=5).fit_transform(X_model,y_model)
X_model_new
print(X_model_new.head())
# np.any(np.isnan(mat))
# np.all(np.isfinite(mat))
# X_model[X_model.isnull()]